# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate):
    return tf.keras.layers.Dropout(rate)

# Create model(class)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))
            
        self.model.add(dense(label_dmi, weight_init))
        
    def call(self, x, training=None, maks=None):
        x = self.model(x)
        return x

# Create model(function)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4):
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.9163, train_loss: 2.32768965, train_accuracy: 0.2656, test_Accuracy: 0.1802
Epoch: [ 0] [    1/  468] time: 1.0143, train_loss: 2.37122941, train_accuracy: 0.2812, test_Accuracy: 0.2288
Epoch: [ 0] [    2/  468] time: 1.1123, train_loss: 2.31170034, train_accuracy: 0.2344, test_Accuracy: 0.2324
Epoch: [ 0] [    3/  468] time: 1.2043, train_loss: 2.37819004, train_accuracy: 0.2109, test_Accuracy: 0.2675
Epoch: [ 0] [    4/  468] time: 1.2933, train_loss: 2.30340981, train_accuracy: 0.3594, test_Accuracy: 0.3523
Epoch: [ 0] [    5/  468] time: 1.3823, train_loss: 2.20549035, train_accuracy: 0.4609, test_Accuracy: 0.3989
Epoch: [ 0] [    6/  468] time: 1.4793, train_loss: 2.12696791, train_accuracy: 0.4531, test_Accuracy: 0.3578
Epoch: [ 0] [    7/  468] time: 1.5703, train_loss: 2.24231577, train_accuracy: 0.3672, test_Accuracy: 0.3571
Epoch: [ 0] [    8/  468] time: 1.6653,

Epoch: [ 0] [   74/  468] time: 7.9952, train_loss: 0.57628095, train_accuracy: 0.8906, test_Accuracy: 0.8818
Epoch: [ 0] [   75/  468] time: 8.0942, train_loss: 0.72338855, train_accuracy: 0.8828, test_Accuracy: 0.8822
Epoch: [ 0] [   76/  468] time: 8.1872, train_loss: 0.69107771, train_accuracy: 0.9062, test_Accuracy: 0.8846
Epoch: [ 0] [   77/  468] time: 8.2802, train_loss: 0.57491326, train_accuracy: 0.8984, test_Accuracy: 0.8892
Epoch: [ 0] [   78/  468] time: 8.3722, train_loss: 0.52302796, train_accuracy: 0.8906, test_Accuracy: 0.8935
Epoch: [ 0] [   79/  468] time: 8.4662, train_loss: 0.60002428, train_accuracy: 0.9297, test_Accuracy: 0.8955
Epoch: [ 0] [   80/  468] time: 8.5662, train_loss: 0.58317447, train_accuracy: 0.8750, test_Accuracy: 0.8957
Epoch: [ 0] [   81/  468] time: 8.6642, train_loss: 0.52373528, train_accuracy: 0.8672, test_Accuracy: 0.8960
Epoch: [ 0] [   82/  468] time: 8.7622, train_loss: 0.73352826, train_accuracy: 0.8750, test_Accuracy: 0.8962
Epoch: [ 0

Epoch: [ 0] [  150/  468] time: 15.3364, train_loss: 0.21850684, train_accuracy: 0.9297, test_Accuracy: 0.9286
Epoch: [ 0] [  151/  468] time: 15.4324, train_loss: 0.30037552, train_accuracy: 0.9453, test_Accuracy: 0.9270
Epoch: [ 0] [  152/  468] time: 15.5294, train_loss: 0.27516514, train_accuracy: 0.9219, test_Accuracy: 0.9277
Epoch: [ 0] [  153/  468] time: 15.6304, train_loss: 0.43148670, train_accuracy: 0.8984, test_Accuracy: 0.9275
Epoch: [ 0] [  154/  468] time: 15.7274, train_loss: 0.31637108, train_accuracy: 0.9609, test_Accuracy: 0.9275
Epoch: [ 0] [  155/  468] time: 15.8274, train_loss: 0.24447279, train_accuracy: 0.9531, test_Accuracy: 0.9266
Epoch: [ 0] [  156/  468] time: 15.9204, train_loss: 0.34477040, train_accuracy: 0.9453, test_Accuracy: 0.9266
Epoch: [ 0] [  157/  468] time: 16.0185, train_loss: 0.23603392, train_accuracy: 0.9141, test_Accuracy: 0.9265
Epoch: [ 0] [  158/  468] time: 16.1195, train_loss: 0.37850526, train_accuracy: 0.8828, test_Accuracy: 0.9269
E

Epoch: [ 0] [  224/  468] time: 22.4956, train_loss: 0.31098479, train_accuracy: 0.9297, test_Accuracy: 0.9340
Epoch: [ 0] [  225/  468] time: 22.5886, train_loss: 0.21819203, train_accuracy: 0.9609, test_Accuracy: 0.9344
Epoch: [ 0] [  226/  468] time: 22.6816, train_loss: 0.24267201, train_accuracy: 0.9375, test_Accuracy: 0.9346
Epoch: [ 0] [  227/  468] time: 22.7866, train_loss: 0.41343006, train_accuracy: 0.9219, test_Accuracy: 0.9354
Epoch: [ 0] [  228/  468] time: 22.8856, train_loss: 0.25248694, train_accuracy: 0.9453, test_Accuracy: 0.9390
Epoch: [ 0] [  229/  468] time: 22.9796, train_loss: 0.39873663, train_accuracy: 0.9453, test_Accuracy: 0.9416
Epoch: [ 0] [  230/  468] time: 23.0756, train_loss: 0.43199989, train_accuracy: 0.9141, test_Accuracy: 0.9414
Epoch: [ 0] [  231/  468] time: 23.1796, train_loss: 0.28503725, train_accuracy: 0.9219, test_Accuracy: 0.9429
Epoch: [ 0] [  232/  468] time: 23.2756, train_loss: 0.28525084, train_accuracy: 0.9766, test_Accuracy: 0.9432
E

Epoch: [ 0] [  297/  468] time: 29.5456, train_loss: 0.27282763, train_accuracy: 0.9297, test_Accuracy: 0.9420
Epoch: [ 0] [  298/  468] time: 29.6486, train_loss: 0.38235039, train_accuracy: 0.9453, test_Accuracy: 0.9419
Epoch: [ 0] [  299/  468] time: 29.7466, train_loss: 0.35680360, train_accuracy: 0.9141, test_Accuracy: 0.9419
Epoch: [ 0] [  300/  468] time: 29.8436, train_loss: 0.27143133, train_accuracy: 0.9375, test_Accuracy: 0.9421
Epoch: [ 0] [  301/  468] time: 29.9366, train_loss: 0.23622914, train_accuracy: 0.9766, test_Accuracy: 0.9426
Epoch: [ 0] [  302/  468] time: 30.0306, train_loss: 0.34302473, train_accuracy: 0.9219, test_Accuracy: 0.9419
Epoch: [ 0] [  303/  468] time: 30.1236, train_loss: 0.52918667, train_accuracy: 0.9141, test_Accuracy: 0.9433
Epoch: [ 0] [  304/  468] time: 30.2284, train_loss: 0.24366346, train_accuracy: 0.9688, test_Accuracy: 0.9446
Epoch: [ 0] [  305/  468] time: 30.3244, train_loss: 0.17090739, train_accuracy: 0.9531, test_Accuracy: 0.9447
E

Epoch: [ 0] [  371/  468] time: 36.6944, train_loss: 0.36766419, train_accuracy: 0.9219, test_Accuracy: 0.9513
Epoch: [ 0] [  372/  468] time: 36.7974, train_loss: 0.28227645, train_accuracy: 0.9688, test_Accuracy: 0.9511
Epoch: [ 0] [  373/  468] time: 36.8894, train_loss: 0.24854128, train_accuracy: 0.9844, test_Accuracy: 0.9514
Epoch: [ 0] [  374/  468] time: 36.9804, train_loss: 0.28800502, train_accuracy: 0.9453, test_Accuracy: 0.9519
Epoch: [ 0] [  375/  468] time: 37.0754, train_loss: 0.27028593, train_accuracy: 0.9609, test_Accuracy: 0.9518
Epoch: [ 0] [  376/  468] time: 37.1754, train_loss: 0.30869213, train_accuracy: 0.9453, test_Accuracy: 0.9516
Epoch: [ 0] [  377/  468] time: 37.2684, train_loss: 0.27054954, train_accuracy: 0.9375, test_Accuracy: 0.9504
Epoch: [ 0] [  378/  468] time: 37.3654, train_loss: 0.27799681, train_accuracy: 0.9375, test_Accuracy: 0.9493
Epoch: [ 0] [  379/  468] time: 37.4634, train_loss: 0.21164155, train_accuracy: 0.9453, test_Accuracy: 0.9492
E

Epoch: [ 0] [  444/  468] time: 43.6295, train_loss: 0.24104942, train_accuracy: 0.9688, test_Accuracy: 0.9504
Epoch: [ 0] [  445/  468] time: 43.7305, train_loss: 0.25662374, train_accuracy: 0.9609, test_Accuracy: 0.9487
Epoch: [ 0] [  446/  468] time: 43.8195, train_loss: 0.23999679, train_accuracy: 0.9609, test_Accuracy: 0.9476
Epoch: [ 0] [  447/  468] time: 43.9135, train_loss: 0.39024699, train_accuracy: 0.9297, test_Accuracy: 0.9476
Epoch: [ 0] [  448/  468] time: 44.0105, train_loss: 0.30839527, train_accuracy: 0.9531, test_Accuracy: 0.9478
Epoch: [ 0] [  449/  468] time: 44.1075, train_loss: 0.26468754, train_accuracy: 0.9453, test_Accuracy: 0.9487
Epoch: [ 0] [  450/  468] time: 44.2015, train_loss: 0.22606796, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  451/  468] time: 44.2915, train_loss: 0.23379813, train_accuracy: 0.9531, test_Accuracy: 0.9522
Epoch: [ 0] [  452/  468] time: 44.3815, train_loss: 0.20373279, train_accuracy: 0.9766, test_Accuracy: 0.9516
E

# Test Accuracy : 96.34%